# This notebook shows the importance of the features introduced in the "EDA about" series below.
- [EDA about time_step and u_out](https://www.kaggle.com/marutama/eda-about-time-step-and-u-out)
- [EDA about u_in](https://www.kaggle.com/marutama/eda-about-u-in)
- [EDA about Pressure Part 1](https://www.kaggle.com/marutama/eda-about-pressure-part-1)
- [EDA about Pressure Part 2](https://www.kaggle.com/marutama/eda-about-pressure-part-2)

The characteristic features are as follows.
- u_in_last : Separate main mode from others
- u_in_mean, u_in_half_mean : Classify pressure layers. It seems that u_in_mean is enough instead of u_in_half_mean.
- area_true : Area_true is higher than the wrong area
- u_in_std, u_in_half_std : Either one is fine
- time_end : Other than the main mode, the mode differs depending on whether it is 2.65s or more or less.
- time_delta : The time interval. The part where the time interval is long seems to be the part where data is missing.
- u_in_uout1 : This is u_in when u_out rises to 1 with this breath_id.
- diff_vib : Indicates the vibration of u_in. The number of times the u_in diff coding is inverted.

If you find it useful, please upvote this notebook and the "EDAabout" series.

Many thanks to Zhangxin-san and Chris Deotte-san for sharing the following two notebooks.
- [finetune of Tensorflow Bidirectional LSTM](https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm)
- [LSTM Feature Importance](https://www.kaggle.com/cdeotte/lstm-feature-importance)



# LSTM Feature Importance (aka permutation importance)
When using an XGB model (gradient boosted trees), we have the advantage of displaying XGB Feature Importance. With Neural Networks, we don't have that advantage. However, we can use a technique called [Permutation Feature Importance][1] to compute the Feature Importance for any model. Therefore we can compute LSTM Feature Importance using permuation importance. Permutation importance has the advantage that we only need to train 1 model versus training multiple models for each feature we wish to evaluate!

Other types of feature importances that can be applied to any model are [SHAP Feature Importance][3] and [LOFO Feature Importance][4].

We will compute LSTM feature importance for Zhangxin's notebook [here][2]. (Thus our notebook here has been forked from his notebook and we will use his saved model weights).

[1]: https://christophm.github.io/interpretable-ml-book/feature-importance.html#feature-importance
[2]: https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm
[3]: https://christophm.github.io/interpretable-ml-book/shap.html
[4]: https://www.kaggle.com/aerdem4/google-ventilator-lofo-feature-importance

In [ ]:
ls ../input/finetune-of-tensorflow-bilstm-eda-about/

# Load Libraries and Data 

In [ ]:
import numpy as np, os
import pandas as pd

import gc

import optuna

# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/274717 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

DEBUG = False
TRAIN_MODEL = False
INFER_TEST = False
ONE_FOLD_ONLY = True
COMPUTE_LSTM_IMPORTANCE = True

if DEBUG:
    train = train[:80*1000]

In [ ]:
%%time
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
pressure_values = np.sort( train.pressure.unique() )
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

# Engineer Features

In [ ]:
def add_features(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()

    #######################################
    # fast area calculation
    df['time_delta'] = df['time_step'].diff()
    df['time_delta'].fillna(0, inplace=True)
    df['time_delta'].mask(df['time_delta'] < 0, 0, inplace=True)
    df['tmp'] = df['time_delta'] * df['u_in']
    df['area_true'] = df.groupby('breath_id')['tmp'].cumsum()
    
    #u_in_max_dict = df.groupby('breath_id')['u_in'].max().to_dict()
    #df['u_in_max'] = df['breath_id'].map(u_in_max_dict)
    #u_in_min_dict = df.groupby('breath_id')['u_in'].min().to_dict()
    #df['u_in_min'] = df['breath_id'].map(u_in_min_dict)
    u_in_mean_dict = df.groupby('breath_id')['u_in'].mean().to_dict()
    df['u_in_mean'] = df['breath_id'].map(u_in_mean_dict)
    del u_in_mean_dict
    u_in_std_dict = df.groupby('breath_id')['u_in'].std().to_dict()
    df['u_in_std'] = df['breath_id'].map(u_in_std_dict)
    del u_in_std_dict
    
    # u_in_half is time:0 - time point of u_out:1 rise (almost 1.0s)
    df['tmp'] = df['u_out']*(-1)+1 # inversion of u_out
    df['u_in_half'] = df['tmp'] * df['u_in']
    
    # u_in_half: max, min, mean, std
    u_in_half_max_dict = df.groupby('breath_id')['u_in_half'].max().to_dict()
    df['u_in_half_max'] = df['breath_id'].map(u_in_half_max_dict)
    del u_in_half_max_dict
    u_in_half_min_dict = df.groupby('breath_id')['u_in_half'].min().to_dict()
    df['u_in_half_min'] = df['breath_id'].map(u_in_half_min_dict)
    del u_in_half_min_dict
    u_in_half_mean_dict = df.groupby('breath_id')['u_in_half'].mean().to_dict()
    df['u_in_half_mean'] = df['breath_id'].map(u_in_half_mean_dict)
    del u_in_half_mean_dict
    u_in_half_std_dict = df.groupby('breath_id')['u_in_half'].std().to_dict()
    df['u_in_half_std'] = df['breath_id'].map(u_in_half_std_dict)
    del u_in_half_std_dict
    
    gc.collect()
    
    # All entries are first point of each breath_id
    first_df = df.loc[0::80,:]
    # All entries are first point of each breath_id
    last_df = df.loc[79::80,:]
    
    # The Main mode DataFrame and flag
    main_df= last_df[(last_df['u_in']>4.8)&(last_df['u_in']<5.1)]
    main_mode_dict = dict(zip(main_df['breath_id'], [1]*len(main_df)))
    df['main_mode'] = df['breath_id'].map(main_mode_dict)
    df['main_mode'].fillna(0, inplace=True)
    del main_df
    del main_mode_dict

    # u_in: first point, last point
    u_in_first_dict = dict(zip(first_df['breath_id'], first_df['u_in']))
    df['u_in_first'] = df['breath_id'].map(u_in_first_dict)
    del u_in_first_dict
    u_in_last_dict = dict(zip(first_df['breath_id'], last_df['u_in']))
    df['u_in_last'] = df['breath_id'].map(u_in_last_dict)
    del u_in_last_dict
    # time(sec) of end point
    time_end_dict = dict(zip(last_df['breath_id'], last_df['time_step']))     
    df['time_end'] = df['breath_id'].map(time_end_dict)
    del time_end_dict
    del last_df
    
    # u_out1_timing flag and DataFrame: speed up
    # 高速版 uout1_df 作成
    df['u_out_diff'] = df['u_out'].diff()
    df['u_out_diff'].fillna(0, inplace=True)
    df['u_out_diff'].replace(-1, 0, inplace=True)
    uout1_df = df[df['u_out_diff']==1]
    
    gc.collect()
    
    #main_uout1 = uout1_df[uout1_df['main_mode']==1]
    #nomain_uout1 = uout1_df[uout1_df['main_mode']==1]
    
    # Register Area when u_out becomes 1
    uout1_area_dict = dict(zip(first_df['breath_id'], first_df['u_in']))
    df['area_uout1'] = df['breath_id'].map(uout1_area_dict)
    del uout1_area_dict
    
    # time(sec) when u_out becomes 1
    uout1_dict = dict(zip(uout1_df['breath_id'], uout1_df['time_step']))
    df['time_uout1'] = df['breath_id'].map(uout1_dict)
    del uout1_dict
    
    # u_in when u_out becomes1
    u_in_uout1_dict = dict(zip(uout1_df['breath_id'], uout1_df['u_in']))
    df['u_in_uout1'] = df['breath_id'].map(u_in_uout1_dict)
    del u_in_uout1_dict
    
    # Dict that puts 0 at the beginning of the 80row cycle
    first_0_dict = dict(zip(first_df['id'], [0]*len(uout1_df)))

    del first_df
    del uout1_df   
    
    gc.collect()
    
    # Faster version u_in_diff creation, faster than groupby
    df['u_in_diff'] = df['u_in'].diff()
    df['tmp'] = df['id'].map(first_0_dict) # put 0, the 80row cycle
    df.iloc[0::80, df.columns.get_loc('u_in_diff')] = df.iloc[0::80, df.columns.get_loc('tmp')]

    # Create u_in vibration
    df['diff_sign'] = np.sign(df['u_in_diff'])
    df['sign_diff'] = df['diff_sign'].diff()
    df['tmp'] = df['id'].map(first_0_dict) # put 0, the 80row cycle
    df.iloc[0::80, df.columns.get_loc('sign_diff')] = df.iloc[0::80, df.columns.get_loc('tmp')]
    del first_0_dict
    
    # Count the number of inversions, so take the absolute value and sum
    df['sign_diff'] = abs(df['sign_diff']) 
    sign_diff_dict = df.groupby('breath_id')['sign_diff'].sum().to_dict()
    df['diff_vib'] = df['breath_id'].map(sign_diff_dict)
    
    if 'diff_sign' in df.columns:
        df.drop(['diff_sign', 'sign_diff'], axis=1, inplace=True)
    if 'tmp' in df.columns:
        df.drop(['tmp'], axis=1, inplace=True)
    
    gc.collect()
    #######################################
    '''
    '''
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    #df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    #df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    #df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    #df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    #df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    #df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    #df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    #df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    #df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    #df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    #df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    #df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    df['cross']= df['u_in']*df['u_out']
    #df['cross2']= df['time_step']*df['u_out']
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    
    df = pd.get_dummies(df)
    return df

In [ ]:
%%time
train = add_features(train)

In [ ]:
%%time
test = add_features(test)

In [ ]:
print('Train dataframe shape',train.shape)
train.head()

In [ ]:
targets = train[['pressure']].to_numpy().reshape(-1, 80)
train.drop(['pressure'], axis=1, inplace=True)
train.drop(['id', 'breath_id'], axis=1, inplace=True)
test = test.drop(['id', 'breath_id'], axis=1)

COLS = ['BASELINE'] + list(train.columns)
print('Number of feature columns =', len(COLS)-1 )

RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)

train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

In [ ]:
np.array(targets).shape

# Compute LSTM Feature Importance
After we train (or load) each fold model, we will compute LSTM feature importance for all of our features. We do this with a for-loop of size `N` where `N` is the number of features we have. For each feature we wish to evaluate, we infer our OOF with that feature column randomly shuffled. If this feature column is important to our LSTM model, then the OOF MAE will become worse for that for-loop step. After our for-loop, we display bars equal to the size of how much MAE worsened without each feature, which is the importance of each feature.

Note that computing LSTM feature importance after each fold will add about 1 minute for every 5 features.

In [ ]:
EPOCH = 300
BATCH_SIZE = 1024
NUM_FOLDS = 10

TPU = False

if TPU:
    # detect and init the TPU
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

    ## instantiate a distribution strategy
    xpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # GET GPU STRATEGY
    xpu_strategy = tf.distribute.get_strategy()

with xpu_strategy.scope():
    kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=2021)
    test_preds = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        #print(fold)
        if (fold != 0) and (ONE_FOLD_ONLY):
            break
        
        K.clear_session()
        
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
        X_train, X_valid = train[train_idx], train[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]
        
        checkpoint_filepath = f"folds{fold}.hdf5"
        if TRAIN_MODEL:
            model = keras.models.Sequential([
                keras.layers.Input(shape=train.shape[-2:]),
                keras.layers.Bidirectional(keras.layers.LSTM(1024, return_sequences=True)),
                keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True)),
                keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)),
                keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
                keras.layers.Dense(128, activation='selu'),
                keras.layers.Dense(1),
            ])
            model.compile(optimizer="adam", loss="mae")

            lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
            es = EarlyStopping(monitor="val_loss", patience=60, verbose=1, mode="min", restore_best_weights=True)
            sv = keras.callbacks.ModelCheckpoint(
                checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
                save_weights_only=False, mode='auto', save_freq='epoch',
                options=None
            )
            model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[lr, es, sv])
            
        else:
            #model = keras.models.load_model('../input/finetune-of-tensorflow-bidirectional-lstm/'+checkpoint_filepath)
            model = keras.models.load_model('../input/finetune-of-tensorflow-bilstm-eda-about/'+checkpoint_filepath)

        if INFER_TEST:
            print(' Predicting test data...')
            test_preds.append(model.predict(test,verbose=0).squeeze().reshape(-1, 1).squeeze())
                    
        if COMPUTE_LSTM_IMPORTANCE:
            results = []
            print(' Computing LSTM feature importance...')

            for k in tqdm(range(len(COLS))):
                if k>0: 
                    save_col = X_valid[:,:,k-1].copy()
                    np.random.shuffle(X_valid[:,:,k-1])
                        
                oof_preds = model.predict(X_valid, verbose=0).squeeze() 
                mae = np.mean(np.abs( oof_preds-y_valid ))
                results.append({'feature':COLS[k],'mae':mae})
        
                if k>0: 
                    X_valid[:,:,k-1] = save_col
         
            # DISPLAY LSTM FEATURE IMPORTANCE
            print()
            df = pd.DataFrame(results)
            df = df.sort_values('mae')
            plt.figure(figsize=(10,20))
            plt.barh(np.arange(len(COLS)),df.mae)
            plt.yticks(np.arange(len(COLS)),df.feature.values)
            plt.title('LSTM Feature Importance',size=16)
            plt.ylim((-1,len(COLS)))
            plt.show()
                               
            # SAVE LSTM FEATURE IMPORTANCE
            df = df.sort_values('mae',ascending=False)
            df.to_csv(f'lstm_feature_importance_fold_{fold}.csv',index=False)
                               
        # ONLY DO ONE FOLD
        #if ONE_FOLD_ONLY:
        #    #break
        #    exit()